### Contents
Teaching a Model AND and OR<br>
&emsp;Create Boolean Data<br>
&emsp;Design a Model<br>
&emsp;Train the Model<br>
&emsp;Test the Model<br>
Teaching a Model XOR<br>
&emsp;Create Boolean Data<br>
&emsp;Design a Model<br>
&emsp;Train the Model<br>
&emsp;Test the Model<br>

In [1]:
import os
import random as rnd
import time
from typing import List, Tuple

import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor

import utilities as util


Create Boolean Data

In [2]:
def create_boolean_data(size: int=1000) -> Tuple:
    '''
    This function will generate a training and validation set for the logical AND and OR functions.
    In other words, each sample will contain two boolean values and the label for each sample will be 
    either (x1 and x2) or (x1 or x2).
    '''
    X = []
    y = []
    count = int(size/2)
    for _ in range(count):
        x1 = rnd.randint(0, 1)
        x2 = rnd.randint(0, 1)
        label_and = x1 and x2
        label_or = x1 or x2
        # Append the and data.
        sample = [x1, x2]
        X.append(sample)
        y.append(label_and)
        # Append the or data.
        X.append(sample)
        y.append(label_or)

    X, y = np.array(X, dtype=np.float32), np.array(y, np.float32)
    X = np.reshape(X, (size, 2))
    y = np.reshape(y, (size, 1))
    return X, y

'''
    #self.X = np.array(X)   #torch.from_numpy(X)
    #self.y = np.array(y)   #torch.from_numpy(y)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, shuffle=True, stratify=y, random_state=42)
    #self.X_valid, self.X_test, self.y_valid, self.y_test = train_test_split(X_remaining, y_remaining, train_size=0.5, shuffle=True, stratify=y_remaining, random_state=42)
    X_train = np.array(X_train)
    X_valid = np.array(X_valid)
    y_train = np.array(y_train)
    y_valid = np.array(y_valid)
    return X_train, X_valid, y_train, y_valid
'''

class BooleanDataset(torch.utils.data.Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index, None], self.y[index, None]

    def __len__(self):
        return len(self.X)

X_train, y_train = create_boolean_data(1000)
print('X_train Dimenstions:', X_train.ndim)
print('X_train Shape:', X_train.shape)
print('X_train type:', X_train.dtype)
print(X_train[:5])

print('y_train Dimenstions:', y_train.ndim)
print('y_train Shape:', y_train.shape)
print('y_train type:', y_train.dtype)
print(y_train[:5])

#X_train, X_valid, y_train, y_valid = create_boolean_data(1000)
#train_dataset = BooleanDataset(X_train, y_train)
#valid_dataset = BooleanDataset(X_valid, y_valid)

#print('X_train Dimenstions:', train_dataset.X.ndim)
#print('X_train Shape:', train_dataset.X.shape)
#print(train_dataset.X[:5])

#print('y_train Dimenstions:', train_dataset.y.ndim)
#print('y_train Shape:', train_dataset.y.shape)
#print(train_dataset.y[:5])

X_train Dimenstions: 2
X_train Shape: (1000, 2)
X_train type: float32
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]]
y_train Dimenstions: 2
y_train Shape: (1000, 1)
y_train type: float32
[[0.]
 [1.]
 [0.]
 [1.]
 [0.]]


Design a Model

In [21]:
class AndOrModel(nn.Module):

    def __init__(self):
        super(AndOrModel, self).__init__()

        self.linear1 = nn.Linear(2, 1, bias=True)

    def forward(self, input):
        l1 = self.linear1(input)
        output = F.sigmoid(l1)
        return output

Create the Model and Print the Initial Parameters

In [22]:
model = AndOrModel()
util.print_parameters(model)

linear1.weight tensor([[0.5406, 0.5869]])
linear1.bias tensor([-0.1657])


Train the Model and Print the Results

In [5]:
config = {
    'batch_size': 2,
    'epochs': 10,
    'lr': 0.01,
    'loss_function': nn.MSELoss()
}

def train_batches(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    total_loss = 0
    for X, y in dataloader:
        # Compute prediction error
        print('X:', X, type(X), X.dtype)
        print('y:', y, type(y), y.dtype)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss_per_batch = total_loss / num_batches
    return avg_loss_per_batch


def validate_batches(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            loss += loss_fn(pred, y).item()
    avg_loss_per_batch = total_loss / num_batches
    return avg_loss_per_batch


def train_model(config, model, train_dataset, valid_dataset):
    start_time = time.time()

    batch_size = config['batch_size']
    epochs = config['epochs']
    loss_fn = config['loss_function']
    lr = config['lr']
    optimizer = optim.SGD(model.parameters(), lr=lr)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=batch_size)

    results = []
    for epoch in range(epochs):
        avg_loss_per_batch = train_batches(train_loader, model, loss_fn, optimizer)
        result = {'train_loss': avg_loss_per_batch}
        avg_loss_per_batch = validate_batches(valid_loader, model, loss_fn)
        result = {'valid_loss': avg_loss_per_batch}
        result['epoch'] = epoch + 1
        result['process_id'] = os.getpid()
        results.append(result)

    duration = time.time() - start_time
    return model, results, duration

In [27]:
# TODO: Fix train_model it is not working.
#model, results, duration = train_model(config, model, train_dataset, valid_dataset)

config = {
    'epochs': 100,
    'lr': 0.01,
    'loss_function': nn.MSELoss()
}

model, losses = util.train_model(model, config, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
util.print_results(model, losses)

0 : 126.08957962400746
10 : 126.08029682119377
20 : 126.0716088162153
30 : 126.06346865516389
40 : 126.0558173043537
50 : 126.04862089114613
60 : 126.0418329979293
70 : 126.0354242491594
80 : 126.02935997804161
90 : 126.02361651181127
99 : 126.01870402850909


In [28]:
util.print_parameters(model)

linear1.weight tensor([[3.8806, 3.8805]])
linear1.bias tensor([-3.8763])


In [ ]:
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_valid]
util.plot_data(np.array(X_train), np.array(y_train), np.array(predictions))

### A Quadratic Example

In [ ]:
def create_quadratic_data(a:float, b: float, c:float) -> Tuple[List[float], List[float]]:
    X = [float(x) for x in range(-10, 11)]
    y = [a*(x**2)+(b*x)+c for x in X]
    X, y = np.array(X, dtype=np.float32), np.array(y, np.float32)
    X = np.reshape(X, (len(X), 1))
    y = np.reshape(y, (len(y), 1))
    return X, y

X_train, y_train = create_quadratic_data(5, 2, 3)

print('X_train Dimenstions:',X_train.ndim)
print('X_train Shape:', X_train.shape)
print(X_train[:2])

print('y_train Dimenstions:',y_train.ndim)
print('y_train Shape:', y_train.shape)
print(y_train[:2])

In [ ]:
plot_data(X_train, y_train)

In [ ]:
model = LinearRegressionModel()
model, losses = train_model(model, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
print_results(model, losses)

In [ ]:
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_train]
plot_data(np.array(X_train), np.array(y_train), np.array(predictions))

In [ ]:
class QuadraticRegressionModel(nn.Module):

    def __init__(self):
        super(QuadraticRegressionModel, self).__init__()

        self.linear1 = nn.Linear(1, 6, bias=True)
        self.linear2 = nn.Linear(6, 6, bias=True)
        self.linear3 = nn.Linear(6, 1, bias=True)

    def forward(self, x, log=False):
        x = F.dropout(F.relu(self.linear1(x)), p=0.5)
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        #out = self.linear1(input)
        #out = F.relu(out)
        #out = F.dropout(out, p=0.5)
        #out = self.linear2(out)
        #out = F.relu(out)
        #out = self.linear3(out)
        return x

Test the Untrained Model with a Single Prediction

In [ ]:
model = QuadraticRegressionModel()
X = torch.tensor([1], dtype=torch.float32)
prediction = model(X, log=True)
print(X)
print(prediction)

In [ ]:
model = QuadraticRegressionModel()
model, losses = train_model(model, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
print_results(model, losses)

Plot labels and predictions

In [ ]:
#model.eval() # Tell the model we are evaluating it so that it does not learn or dropout.
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_train]

plot_data(np.array(X_train), np.array(y_train), np.array(predictions))